### Categorize kanji

In [8]:
# imports
import pandas

In [9]:
# read excel file (with mapping)
df = pandas.read_excel("mbo.xlsx")
df.columns = ["CHAR", "COMPONENTS1", "COMPONENTS2", "COMPONENTS3", "COMPONENTS4", "COMPONENTS5", "COMPONENTS6", "ON READING", "KUN READING", "KEYWORD", "SRL", "TYPE", "FREQ"]
df

,CHAR,COMPONENTS1,COMPONENTS2,COMPONENTS3,COMPONENTS4,COMPONENTS5,COMPONENTS6,ON READING,KUN READING,KEYWORD,SRL,TYPE,FREQ
0,一,NaN,NaN,NaN,NaN,NaN,NaN,イチ、イツ,ひと・つ,one,5,STEM,10.800000
1,二,NaN,NaN,NaN,NaN,NaN,NaN,ニ,ふた・つ,two,5,MEAN,128.300000
2,三,NaN,NaN,NaN,NaN,NaN,NaN,サン,みっ・つ,three,5,MEAN,120.700000
3,四,NaN,NaN,NaN,NaN,NaN,NaN,シ,よっ・つ、よん、よ,four,5,MEAN,312.073333
4,五,NaN,NaN,NaN,NaN,NaN,NaN,ゴ,いつ・つ,five,5,MEAN,315.626667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,気,气,㐅,NaN,NaN,NaN,NaN,キ、ケ,NaN,atmosphere,1,OTHER,64.000000
460,風,𠘨,䖝,NaN,NaN,NaN,NaN,フウ、フ,かぜ,1 wind 2 style,1,OTHER,289.700000
461,乳,⺤,子,乚,NaN,NaN,NaN,ニュウ,ち、ちち,milk,1,OTHER,1067.000000
462,興,臼,同,ハ,NaN,NaN,NaN,コウ、キョウ,NaN,interest,1,OTHER,649.600000


In [6]:
# rule 1&2 (simple filter)

# filter keywords
list_keywords = df["KEYWORD"].to_list()
# list_keywords

### Example - 脈

In [52]:
example = df[df["CHAR"] == '脈']
example

,CHAR,COMPONENTS1,COMPONENTS2,COMPONENTS3,COMPONENTS4,COMPONENTS5,COMPONENTS6,ON READING,KUN READING,KEYWORD,SRL,TYPE,FREQ
80,郷,幺,艮,阝,NaN,NaN,NaN,キョウ,NaN,hometown,1,VR,1029.706667


In [53]:
def find_components(row, df):
    component1 = row["COMPONENTS1"].to_list()[0]
    return df[(df["COMPONENTS1"] == component1) | (df["COMPONENTS2"] == component1) | (df["COMPONENTS3"] == component1)]

find_components(example, df)

# df[df["COMPONENTS1"] == "c"]
# mb = mb.dropna(subset=['ON READING'])
# mb[mb["ON READING"].str.ns("フク")]

,CHAR,COMPONENTS1,COMPONENTS2,COMPONENTS3,COMPONENTS4,COMPONENTS5,COMPONENTS6,ON READING,KUN READING,KEYWORD,SRL,TYPE,FREQ
80,郷,幺,艮,阝,NaN,NaN,NaN,キョウ,NaN,hometown,1,VR,1029.706667
